In [1]:
import requests
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import re

# PREMISE

I'm sure we all know someone who's go to excuse is "[insert team here] vs Refs", or a sarcastic "I wonder who Vegas has in this one", when their favorite team loses a game. I know its hard to accept when your team has been flat out beat, but with data science we can actually analyze and see if refs tend to favorite the Vegas-backed team, and if refs really do impact games in significant way.

What we are going to do in this tutorial to start, is scrape data on NFL games - which referee reffed it, the moneyline and other betting lines, the play-by-play, who was home and who was away, etc. 

Once we get all the data we need in a tidy manner, and analyze it, we will be able to answer a lot of questions about refs tendencies, and just the overall patterns we can find regarding penalties, and you will be able to tell that friend either to just accept the loss, or that he has a legitimate claim. 

Some good questions to look at are
1. When the favorited team goes behind, do the refs call more penalties on the opposing team
2. Do certain refs tend to favor certain calls, or certain teams
3. ...

## DATA COLLECTION

What we want to do is first collect all the data we need, via webscraping with BeautifulSoup and Requests, and then storing all the data in a Pandas dataframe.

Here, I have already downloaded an excel sheet from https://www.sportsbookreviewsonline.com/scoresoddsarchives/nfl/nfloddsarchives.html

We read it into an excel-sheet like data type called a Dataframe, from the Pandas module. (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)

In [2]:
gamedata = pd.read_excel('nflodds.xlsx')
gamedata

,Date,Rot,VH,Team,1st,2nd,3rd,4th,Final,Open,Close,ML,2H
0,909,451,V,Dallas,7,9,10,3,29,52.5,52.5,375,27
1,909,452,H,TampaBay,7,14,7,3,31,7,10,-450,2
2,912,453,V,Pittsburgh,0,0,6,17,23,51,47.5,240,0.5
3,912,454,H,Buffalo,3,7,0,6,16,7,7,-280,21.5
4,912,455,V,NYJets,0,0,8,6,14,43.5,44.5,160,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,130,322,H,KansasCity,7,14,0,3,24,7,7,-310,3
566,130,323,V,SanFrancisco,0,10,7,0,17,47.5,45.5,170,23.5
567,130,324,H,LARams,0,7,0,13,20,3,-3.5,-180,0.5
568,213,101,N,LARams,7,6,3,7,23,4,4,-200,0.5


This data contains information on games, and the moneyline, opening, halftime, and closing lines for a game.

What we now want to do now is scrape data for the play-by-play so we can see exactly when every single flag was thrown. The website we are scraping from, https://www.nflpenalties.com/all-referees.php?year=2021, contains links to each referees individual page. To analyze webpages, you don't need to know anything about HTML except how to  press right-click inspect and look through the HTML for what you want, and then utilize BeautifulSoup (https://www.crummy.com/software/BeautifulSoup/bs4/doc/) and Regular Expressions to extract the information you want. BeautifulSoup parses the content returned by a request sent through the requests module, and turns it into something that you can use to search for specific elements of HTML.

In [3]:
URL = 'https://www.nflpenalties.com/all-referees.php?year=2021'
PREFIX = 'https://www.nflpenalties.com'

ref_to_games = {}

# get all links to the referees
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')
elts = soup.find_all(href=True)

# compile regex for use in loop
match_referee = re.compile(r'<a href="(/referee/.*)">(.*)<')

for elt in elts: 
    # if the link is a link to a referee, add it to the mapping
    if ref := re.match(match_referee, str(elt)): # love the := operator 
        ref_to_games[ref.group(2)] = PREFIX+ref.group(1)
        
ref_to_games['Adrian Hill']

'https://www.nflpenalties.com/referee/adrian-hill?year=2021'

Throughout this tutorial, I will be using Adrian Hill as my example instead of printing every single ref.

Utilizing BeautifulSoup, the re module, and requests, we create a python dictionary, ref_to_games, that maps a referee to their webpage's link, as shown above.

From there, we then have to search through each link and see where that takes us. Looking at the HTML for the pages, I noticed that there is a link to every single game refereed by that ref. From there, we now the next step is to extract those links for each referee.

In [4]:
# compile regex for use in loop
match_game = re.compile(r'<a href="(/game/.*)">')

for ref, link in ref_to_games.items():
    games = []

    # get all links to the games for each referee
    page = requests.get(link)
    soup = BeautifulSoup(page.content, 'html.parser')
    elts = soup.find_all(href=True)

    for elt in elts: 
        # if the link is a link to a game, add it to the list for that ref
        if game := re.match(match_game, str(elt)):
            games.append(PREFIX+game.group(1))
            
    ref_to_games[ref] = games
    
ref_to_games['Adrian Hill']

['https://www.nflpenalties.com/game/minnesota-vikings-at-cincinnati-bengals-09-12-2021',
 'https://www.nflpenalties.com/game/detroit-lions-at-green-bay-packers-09-20-2021',
 'https://www.nflpenalties.com/game/atlanta-falcons-at-new-york-giants-09-26-2021',
 'https://www.nflpenalties.com/game/tennessee-titans-at-new-york-jets-10-03-2021',
 'https://www.nflpenalties.com/game/kansas-city-chiefs-at-washington-football-team-10-17-2021',
 'https://www.nflpenalties.com/game/denver-broncos-at-cleveland-browns-10-21-2021',
 'https://www.nflpenalties.com/game/jacksonville-jaguars-at-seattle-seahawks-10-31-2021',
 'https://www.nflpenalties.com/game/houston-texans-at-miami-dolphins-11-07-2021',
 'https://www.nflpenalties.com/game/carolina-panthers-at-arizona-cardinals-11-14-2021',
 'https://www.nflpenalties.com/game/chicago-bears-at-detroit-lions-11-25-2021',
 'https://www.nflpenalties.com/game/washington-football-team-at-las-vegas-raiders-12-05-2021',
 'https://www.nflpenalties.com/game/seattle-s

In this code, we have updated the ref_to_games dictionary to map each referee to all links to the play-by-plays for the games that they refereed, as shown above.

Now, we have to go to through those links, and extract the play-by-play for each game.

In [5]:
for ref, links in ref_to_games.items():
    games = []
    for link in links:
        games.append(pd.read_html(link, attrs={'class':'standard'})[0])
    ref_to_games[ref] = games
    
ref_to_games['Adrian Hill'][0]

,MIN,CIN,Poss,Time,Down,Distance,Yardline,Play
0,0,0,CIN,Q1 15:00,0,0,CIN 35,2-E.McPherson kicks 65 yards from CIN 35 to en...
1,0,0,MIN,Q1 15:00,1,10,MIN 25,"(15:00) PENALTY on MIN-30-C.Ham, False Start, ..."
2,0,0,MIN,Q1 15:00,1,15,MIN 20,(15:00) 8-K.Cousins pass short right to 33-D.C...
3,0,0,MIN,Q1 14:25,2,6,MIN 29,"(14:25) PENALTY on MIN-83-T.Conklin, False Sta..."
4,0,0,MIN,Q1 14:10,2,11,MIN 24,(14:10) 8-K.Cousins pass short middle to 83-T....
...,...,...,...,...,...,...,...,...
235,24,24,CIN,Q5 00:06,2,5,MIN 15,(:06) 9-J.Burrow spiked the ball to stop the c...
236,24,24,NaN,Q5 00:05,0,0,NaN,Timeout #1 by MIN at 00:05.
237,24,27,CIN,Q5 00:05,3,5,MIN 15,(:05) 2-E.McPherson 38 yard field goal is GOOD...
238,MIN,CIN,Poss,Time,Down,Distance,Yardline,Play


These few lines of code take a while to run, upwards of 2 minutes on my laptop to iterate through every NFL game played in a season, but after they run we have successfully updated ref_to_games dictionary to map referees to a list of Dataframes of play-by-play information for each game the referee coached. At this point we have finished scraping our data, and we have successfully stored the play-by-play for every single game in the season.

Printed above is the first game refereed by Adrian Hill in the 2021 season, ```ref_to_games[name][i]``` corresponds to the ```(i+1)th game``` coached by ```name```

## DATA PROCESSING

We can modify the rot column in the gamedata frame to create a unique, identifying variable for each matchup.

In [6]:
ids = [(rot-1) if rot%2==0 else rot for rot in gamedata['Rot']]
del gamedata['Rot']
gamedata['ID'] = ids

While iterating through the play-by-plays, I found that one of the dataframes is incorrectly formatted.

In [7]:
ref_to_games['John Hussey'][3]

,0,1,2,3,4,5,6,7
0,0,0,NaN,Q1,0,0,NaN,The game has been suspended. Kickoff has been ...
1,0,0,NaN,Q1,0,0,NaN,The game has resumed.
2,LV,LAC,Poss,Time,Down,Distance,Yardline,Play
3,0,0,LV,Q1 15:00,0,0,LV 35,2-D.Carlson kicks 65 yards from LV 35 to end z...
4,0,0,LAC,Q1 15:00,1,10,LAC 25,(15:00) (Shotgun) 10-J.Herbert pass short left...
...,...,...,...,...,...,...,...,...
200,14,28,LV,Q4 01:15,4,7,LV 13,(1:15) (Shotgun) 4-D.Carr pass incomplete shor...
201,LV,LAC,Poss,Time,Down,Distance,Yardline,Play
202,14,28,LAC,Q4 01:11,1,10,LV 13,(1:11) 10-J.Herbert kneels to LV 14 for -1 yards.
203,14,28,LAC,Q4 00:31,2,11,LV 14,(:31) 10-J.Herbert kneels to LV 15 for -1 yards.


As you can see, the labels for the dataframe are incorrectly formatted. We must tidy up this dataframe so that our iteration below will work.

In [8]:
ref_to_games['John Hussey'][3].columns = ref_to_games['Adrian Hill'][0].columns # adrian hills dataframe is correctly formatted

With this sorted, we can begin the rest of the tidying. 

We are only interested in penalties - not 5 yard incomplete checkdowns. So, we have to drop all rows in each game that does not detail a penalty

In [9]:
# compile regex for use in loop
penalty_pat = re.compile(r'PENALTY')

for ref, games in ref_to_games.items():
    for i, game in enumerate(games):
        
        # remove plays that dont contain penalty information
        filter = game['Play'].str.contains(penalty_pat)
        ref_to_games[ref][i] = game[filter]
    
ref_to_games['Adrian Hill'][0]

,MIN,CIN,Poss,Time,Down,Distance,Yardline,Play
1,0,0,MIN,Q1 15:00,1,10,MIN 25,"(15:00) PENALTY on MIN-30-C.Ham, False Start, ..."
3,0,0,MIN,Q1 14:25,2,6,MIN 29,"(14:25) PENALTY on MIN-83-T.Conklin, False Sta..."
7,0,0,MIN,Q1 12:30,2,9,MIN 43,"(12:30) PENALTY on MIN-30-C.Ham, False Start, ..."
8,0,0,MIN,Q1 12:16,2,14,MIN 38,(12:16) 8-K.Cousins pass short middle to 19-A....
55,0,0,MIN,Q2 09:33,2,13,CIN 25,"(9:33) PENALTY on MIN-17-K.Osborn, False Start..."
56,0,0,MIN,Q2 09:08,2,18,CIN 30,(9:08) (Shotgun) 8-K.Cousins pass incomplete s...
61,0,0,MIN,Q2 07:08,2,9,CIN 10,(7:08) 8-K.Cousins sacked at CIN 20 for -10 ya...
73,7,0,CIN,Q2 02:02,1,10,MIN 29,(2:02) (Shotgun) 9-J.Burrow pass incomplete de...
81,7,7,MIN,Q2 01:44,2,10,MIN 25,(1:44) (Shotgun) 8-K.Cousins pass short middle...
94,7,14,MIN,Q2 00:35,1,10,MIN 25,"(:35) (Shotgun) PENALTY on MIN-75-B.O'Neill, F..."


Now that we have only the plays that are penalties, we can do some tidying up, regarding the data in the columns.

In [16]:
# compile regex for use in loop
penalty_on_pat = re.compile(r'PENALTY on (\w+)-')
penalty_pat = re.compile(r'.*,(.*),.*yard.*')
time_pat = re.compile(r'Q(\d)(.*)')

for ref, games in ref_to_games.items():
    for i, game in enumerate(games):

        # initialize new columns
        penalties_on = []
        penalties = []
        quarters = []
        times = []
        refs = []

        # using regexes to extract data from rows
        for _, row in game.iterrows():
            penalties_on.append(re.match(penalty_on_pat, row['Play']).group(1))
            penalties.append(re.match(penalty_pat, row['Play']).group(1))
            time_match = re.match(time_pat, row['Time'])
            quarters.append(time_match.group(1))
            times.append(time_match.group(2))
            refs.append(ref)

        # adding new columns 
        ref_to_games[ref][i] = game.drop(labels=['Time'], axis='columns') 
        ref_to_games[ref][i].insert(0, 'Ref', refs)
        ref_to_games[ref][i].insert(0, 'Time', times)
        ref_to_games[ref][i].insert(0, 'Quarter', quarters)
        ref_to_games[ref][i].insert(0, 'Called On', penalties_on)
        ref_to_games[ref][i].insert(0, 'Penalty', penalties)

(15:00) PENALTY on MIN-30-C.Ham, False Start, 5 yards, enforced at MIN 25 - No Play.
(14:25) PENALTY on MIN-83-T.Conklin, False Start, 5 yards, enforced at MIN 29 - No Play.
(12:30) PENALTY on MIN-30-C.Ham, False Start, 5 yards, enforced at MIN 43 - No Play.
(12:16) 8-K.Cousins pass short middle to 19-A.Thielen to MIN 42 for 4 yards (55-L.Wilson) [65-L.Ogunjobi]. PENALTY on MIN-74-O.Udoh, Offensive Holding, 10 yards, enforced at MIN 38 - No Play.
(9:33) PENALTY on MIN-17-K.Osborn, False Start, 5 yards, enforced at CIN 25 - No Play.
(9:08) (Shotgun) 8-K.Cousins pass incomplete short middle to 19-A.Thielen (24-V.Bell). PENALTY on MIN-56-G.Bradbury, Offensive Holding, 10 yards, enforced at CIN 30 - No Play.
(7:08) 8-K.Cousins sacked at CIN 20 for -10 yards (65-L.Ogunjobi). PENALTY on CIN-20-E.Apple, Defensive Holding, 5 yards, enforced at CIN 10 - No Play.
(2:02) (Shotgun) 9-J.Burrow pass incomplete deep right to 85-T.Higgins. PENALTY on MIN-21-B.Breeland, Defensive Pass Interference, 26 